## Map Mutations to 3D Structures in the Protein Data Bank

In [1]:
from mmtfPyspark.datasets import g2sDataset, pdbjMineDataset
from ipywidgets import interact, IntSlider
import py3Dmol
import pandas as pd

#### Input parameters

In [2]:
cutoff_distance = 8 # cutoff distance for finding and visualizing interactions
input_file_name = 'mutations.csv' # mutations in standard format (e.g., chr5:g.149440497C>T)
output_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures

## Read 'mutations.csv' file created in the previous step

In [3]:
df = pd.read_csv(input_file_name)

## Create a list of the variants

In [4]:
var_ids = df['var_id'].tolist()
var_ids

['chr6:g.52619766C>T', 'chr11:g.5246945G>T']

## Map the mutations from genome locations to 3D PDB positions¶
Here we use [g2sDataset](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/g2sDataset.py) to retrieve genome to PDB mapping information using the [G2S](https://g2s.genomenexus.org/) (Genome to Structure) web services.

Note, this step may take several minutes to complete.

In [5]:
pdb_map = g2sDataset.get_position_dataset(var_ids, ref_genome='hgvs-grch37').toPandas()
pdb_map = pdb_map.drop_duplicates(subset=['pdbPosition','pdbAminoAcid','variationId'])
pdb_map

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId
0,1GUH,A,83,G,hgvs-grch37,chr6:g.52619766C>T
1,1XZ7,B,108,N,hgvs-grch37,chr11:g.5246945G>T
3,3KMF,G,708,N,hgvs-grch37,chr11:g.5246945G>T
8,3KMF,C,308,N,hgvs-grch37,chr11:g.5246945G>T
12,1AGS,B,82,R,hgvs-grch37,chr6:g.52619766C>T
32,1DXT,B,109,N,hgvs-grch37,chr11:g.5246945G>T


One can also get the mapping for a specific PDB chain, e.g.,

`pdb_map = g2sDataset.get_position_dataset(var_ids, '6H25', 'J', ref_genome='hgvs-grch37').toPandas()`

## Filter PDB Chains

Here we use the SIFTS annotation provided by EBI to filter by taxonomy. To learn more about how to [retrieve SIFTS annotation](
https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/demos/datasets/SiftsDataDemo.ipynb).

#### To filter by organism, we retrieve the SIFTS taxonomy annotations on a PDB chain level
Here we are only interested in human proteins.

In [6]:
taxonomyQuery = "SELECT * FROM sifts.pdb_chain_taxonomy WHERE sifts.pdb_chain_taxonomy.scientific_name = 'Homo sapiens'"
taxonomy = pdbjMineDataset.get_dataset(taxonomyQuery).toPandas()
taxonomy.head()

,pdbid,chain,tax_id,scientific_name,structureChainId
0,10GS,A,9606,Homo sapiens,10GS.A
1,10GS,B,9606,Homo sapiens,10GS.B
2,11GS,A,9606,Homo sapiens,11GS.A
3,11GS,B,9606,Homo sapiens,11GS.B
4,121P,A,9606,Homo sapiens,121P.A


## Filter the PDB chains by joining with the taxonomy data

In [7]:
pd.options.display.max_columns = None # show all rows
pdb_filtered = pdb_map.merge(taxonomy, left_on=['structureId','chainId'], right_on=['pdbid','chain'], how='inner')
pdb_filtered

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId,pdbid,chain,tax_id,scientific_name,structureChainId
0,1GUH,A,83,G,hgvs-grch37,chr6:g.52619766C>T,1GUH,A,9606,Homo sapiens,1GUH.A
1,1XZ7,B,108,N,hgvs-grch37,chr11:g.5246945G>T,1XZ7,B,9606,Homo sapiens,1XZ7.B
2,3KMF,G,708,N,hgvs-grch37,chr11:g.5246945G>T,3KMF,G,9606,Homo sapiens,3KMF.G
3,3KMF,C,308,N,hgvs-grch37,chr11:g.5246945G>T,3KMF,C,9606,Homo sapiens,3KMF.C
4,1DXT,B,109,N,hgvs-grch37,chr11:g.5246945G>T,1DXT,B,9606,Homo sapiens,1DXT.B


## Save mappings

In [8]:
pdb_filtered.to_csv(output_file_name, index=False)

## View mutations grouped by protein chain
Use the slider to view each protein chain.

In [9]:
chains = pdb_filtered.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['1DXT.B', '1GUH.A', '1XZ7.B', '3KMF.C', '3KMF.G']


In [10]:
def view_grouped_mutations(grouped_df, cutoff_distance, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

#       # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

#       # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
       
        # highlight chain of interest in green
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        rows = group.shape[0]
        for j in range(0, rows):
            res_num = str(group.iloc[j]['pdbPosition'])
            mod_res = {'resi': res_num, 'chain': chain_id} 
            col = 'red'
            c_col = col + 'Carbon'
            viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
            viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})          
            if show_labels:
                label = group.iloc[j]['variationId']
                viewer.addLabel(label, {'fontSize':7,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})
        
        
        #print header
        print("PDB Id: " + pdb_id + " chain Id: " + chain_id)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])

        viewer.zoomTo({'chain': chain_id})
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

Turn off scrolling in the viewer cell below

In [11]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

<IPython.core.display.Javascript object>

In [12]:
view_grouped_mutations(chains, 8);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

## View one mutation at a time
Use the slider to view each mutation. Surrounding residues within the `cutoff_distance` are rendered as orange sticks.

In [13]:
def view_single_mutation(df, cutoff_distance, *args):

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

        # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
       
        # highlight chain of interest in green
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'opacity':0.5}})
        
        res_num = str(df.iloc[i]['pdbPosition'])
        label = df.iloc[i]['variationId']
        
        mod_res = {'resi': res_num, 'chain': chain_id} 
        col = 'red'
        c_col = col + 'Carbon'
        viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
        viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.8}})          
        
        if show_labels:
            viewer.addLabel(label, {'fontSize':12,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})
           
        #print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "mutation:", label)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])

        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': cutoff_distance}
        
        # residues surrounding mutation positions
        viewer.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
        
        viewer.zoomTo(surroundings)
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

In [14]:
view_single_mutation(pdb_filtered, cutoff_distance);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

## Now run the next step
Map mutations occuring at protein-polymer interfaces: [3-MapToPolymerInteractions.ipynb](3-MapToPolymerInteractions.ipynb)